# ISMIR 2021 -- Scales, chords, and cadences: Practical music theory for MIR Researchers
# Session 1 -- Scales

***
## 1 Scale Identification (Key Finding) -- Major/Minor
### Music21 -- Symbolic Music
Import libraries.

In [14]:
from music21 import *              # analysis package for symbolic data
import numpy as np                 # matrix computing
import librosa                     # music/audio package 
import librosa.display             # for librosa chroma plot
import matplotlib.pyplot as plt    # plotting
import seaborn as sns              # data visualization based on matplotlib
from collections import deque      # rotate function
import IPython.display as ipd      # audio playback
import pyACA                       # audio content analysis
#from IPython.display import Image  # display png files
from scipy.stats import pearsonr   # Pearson correlation

ModuleNotFoundError: No module named 'pyACA'

#### 1.1 Example 1 -- TAVERN, K. 573
Let's import the opening phrase from the first movement of Mozart's K. 573, encoded in .krn in the TAVERN data set. (You can download TAVERN here: <a href="https://github.com/jcdevaney/TAVERN" target="_blank">https://github.com/jcdevaney/TAVERN</a>.)

In [4]:
theme1 = converter.parse("M573_00_01a_a.krn")

NameError: name 'converter' is not defined

Now let's visualize the score using Music21.

In [ ]:
theme1.show()

One typical approach to identifying the operative key/scale is to compare a 0th-order distribution of pitch classes to the Krumhansl-Kessler (1982) key profiles. Here's the major-key profile for C major.

In [ ]:
# KS algorithm.
ks = analysis.discrete.KrumhanslSchmuckler()
maj = ks.getWeights('major')
min1 = ks.getWeights('minor')
pcs = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# Plot KS major-key figure.
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.plot(pcs,maj)
ax.plot(pcs,min1)
ax.set_ylabel('Degree of Fit')
ax.set_xlabel('Pitch Classes')
plt.ylim([1,7])
plt.title('KS Major-Key Profile (1982)')
plt.rcParams.update({'font.size': 14})
plt.show()

Now let's compute the distribution of pitch classes found in the Mozart theme.

In [ ]:
# Count PCs.
cts = {'C':0, 'C#':0, 'D':0, 'D#':0, 'E':0, 'F':0, 'F#':0, 'G':0, 'G#':0, 'A':0, 'A#/Bb':0, 'B':0}
PC = analysis.pitchAnalysis.pitchAttributeCount(theme1,'name')
cts.update(PC)

# Plot.
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(cts.keys(),cts.values())
ax.set_ylabel('Count')
ax.set_xlabel('Pitch Classes')
plt.title('PC Histogram')

Unsurprisingly, the PC content emphasizes scale-degrees associated with the D major scale. The KS algorithm identifies the key/scale of an excerpt by correlating the major and minor-key profiles for every starting pitch class with the distribution of pitch classes in the excerpt. Let's take this approach for our example and print the correlation coefficient for the most correlated key.

In [ ]:
key = theme1.analyze('key.krumhanslschmuckler')
print(key)
key.correlationCoefficient

Manual calculation of the correlation coefficient.

In [ ]:
val = list(cts.values())
val = np.roll(tmp,-2) # shift to D major at starting position
cor = pearsonr(maj,val)
print('D major')
print(cor[0])

The KS algorithm matches our intuitions! Here's the D major scale for comparison.

In [ ]:
m1 = stream.Measure()
m1.timeSignature = meter.TimeSignature('4/4')
m1.keySignature = key.KeySignature(2)
m1.append([note.Note('D'), note.Note('E'), note.Note('F#'), note.Note('G')])
m2 = stream.Measure()
m2.append([note.Note('A'),note.Note('B'),note.Note('C#5'),note.Note('D5')])
p = stream.Part()
p.append([m1, m2])
p.show()

#### 1.2 Example 2 -- TAVERN, K. 501
What about for an excerpt that modulates from one key to another? Let's look at the opening phrase from Mozart, K. 501, which tonicizes (or modulates to) the key of the dominant.

In [ ]:
theme2 = converter.parse("M501_00_01a_a.krn")
theme2.show()

What's the key across the entire excerpt?

In [ ]:
# Count PCs.
cts2 = {'C':0, 'C#':0, 'D':0, 'D#':0, 'E':0, 'F':0, 'F#':0, 'G':0, 'G#':0, 'A':0, 'A#/Bb':0, 'B':0}
PC = analysis.pitchAnalysis.pitchAttributeCount(theme2,'name')
cts2.update(PC)

# KS algorithm
key = theme2.analyze('key.krumhanslschmuckler')
print(key)
key.correlationCoefficient

The excerpt begins in G major, but the KS algorithm predicts that the entire excerpt is in D major instead. We can address this issue by performing a windowed analysis using the KS algorithm.

In [ ]:
# Key finding over windows of various sizes (1-12 quarter-note beats).
p = graph.plot.WindowedKey(theme2)
p.run()

***
## 2 Scale Identification (Key Finding) -- Major/Minor 
### librosa -- Audio
#### 2.1 Example 1 -- TAVERN, K. 573
Sanity check: Let's try all of this again for the audio representation. You can get the audio for the first example here: https://imslp.org/wiki/9_Variations_on_a_Minuet_by_Duport%2C_K.573_(Mozart%2C_Wolfgang_Amadeus)

The windowed analysis is better, but still doesn't really conform to musical intuitions. Perhaps listeners use more than 0th-order statistics to determine the key?

Play the audio file.

In [ ]:
file = 'K573_themeA.wav'
sig , sr = librosa.load(file,mono=True,sr=None)

Separate the harmonic and percussive components.

In [ ]:
ipd.Audio(file) 

Now let's estimate a chromagram using librosa.

In [ ]:
sig_harmonic, sig_percussive = librosa.effects.hpss(sig)
chroma = librosa.feature.chroma_cqt(sig_harmonic,sr)

Plot the chroma vector and compare it with the PC distribution we plotted earlier.

In [ ]:
plt.figure(figsize=(15, 4))
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Power spectrum chromagram')
plt.tight_layout()
plt.show()

Let's compare this chroma vector to the KS major- and minor-key profiles using pyACA. (Note that pyACA uses a distance metric (Manhattan distance) instead of a correlation coefficient, so we'll have to compute the Pearson correlation manually.)

In [ ]:
# Calculate average energy in each chroma bin.
chroma_mean = np.mean(chroma,axis=1)

# Replot PC histogram from symbolic representation.
fig, axs = plt.subplots(1,2,figsize=(15,5))
axs[0].bar(cts.keys(),cts.values())
axs[0].set_ylabel('Count')
axs[0].set_xlabel('Pitch Classes')
axs[0].set_title('Symbolic')

# Plot Chroma histogram from audio representation.
axs[1].bar(pcs,chroma_mean)
axs[1].set_ylabel('Proportion')
axs[1].set_xlabel('Pitch Classes')
axs[1].set_title('Audio')

#### 2.2 Example 1 -- TAVERN, K. 501

In [ ]:
# Estimate key using pyACA.
key = pyACA.computeKey(sig_harmonic,sr)
print(key[0])

# Correlate manually in order to compare with the symbolic example.
chromas = np.roll(chroma_mean,-2) # shift to D major at starting position
cor = pearsonr(maj,chromas)
print(cor[0])

How will this method fare for K 501? (There isn't a CC license for K. 501 on imslp.org, so the audio recording is not included in the tutorial, but I'll use my own copy for demonstration purposes.) 

#### 1.2.2 Example 2 -- TAVERN, K. 501

In [ ]:
# Import file.
file = 'K501_themeA.wav'
sig, sr = librosa.load(file,mono=True,sr=None)

# Separate harmonic and percussive components.
sig_harmonic, sig_percussive = librosa.effects.hpss(sig)
chroma = librosa.feature.chroma_cqt(sig_harmonic,sr)

# Calculate average energy in each chroma bin.
chroma_mean = np.mean(chroma,axis=1)

# Replot PC histogram from symbolic representation.
fig, axs = plt.subplots(1,2,figsize=(15,5))
axs[0].bar(cts2.keys(),cts2.values())
axs[0].set_ylabel('Count')
axs[0].set_xlabel('Pitch Classes')
axs[0].set_title('Symbolic')

# Plot Chroma histogram from audio representation.
axs[1].bar(pcs,chroma_mean)
axs[1].set_ylabel('Proportion')
axs[1].set_xlabel('Pitch Classes')
axs[1].set_title('Audio')

Clearly key-finding for the audio representation is more challenging!

***

## 3 Scale Identification (Key Finding) -- Beyond Major/Minor
### librosa -- Audio
#### 3.1 The Who

These problems compound when we look beyond common-practice music. Many of the popular music traditions encountered on the radio reflect other scale systems. The Who's "Won't Get Fooled Again" is one such example. Let's listen to an excerpt. (There is no CC license for this song, so I haven't included the file in the tutorial, but I'll play it here.)

In [ ]:
# Estimate key using pyACA.
key = pyACA.computeKey(sig_harmonic,sr)
print(key[0])

# Correlate manually in order to compare with the symbolic example.
chromas = np.roll(chroma_mean,-11) # shift to G major at starting position
cor = pearsonr(min1,chromas)
print(cor[0])

In [ ]:
who = 'TheWho_WontGetFooledAgain.mp3'
sig,sr = librosa.load(file,mono=True,sr=None,duration=60)

In [ ]:
# Import file.
sig, sr = librosa.load('TheWho_WontGetFooledAgain.wav',mono=True,sr=None)

Play the audio file.

In [ ]:
ipd.Audio(sig,rate=sr)

Now let's estimate a chromagram using librosa.

Visualize the chroma vector.

In [ ]:
# Separate harmonic and percussive components.
sig_harmonic, sig_percussive = librosa.effects.hpss(sig)
chroma = librosa.feature.chroma_cqt(sig_harmonic,sr)

# Calculate average energy in each chroma bin.
chroma_mean = np.mean(chroma,axis=1)

# Replot PC histogram from symbolic representation.
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(pcs,chroma_mean)
ax.set_ylabel('Proportion')
ax.set_xlabel('Chromas')
ax.set_title('The Who -- Won''t Get Fooled Again')

#### 3.2 Annotations -- RS-200
Rather than work from the audio representation, let's look at the chord annotations for this song in the RollingStone-200 data set: http://rockcorpus.midside.com/

In [ ]:
text_file = open('wont_get_fooled_again_dt.har')
file_content = text_file.read()
print(file_content)

Here's a plot of the scale-degree content extracted from the chord labels in the RollingStone-200 data set for this song.

In [ ]:
Image(filename='TheWho_SDdistribution.png',width=500,height=500)

This looks like the mixolydian mode! What about if we only count scale degrees if they appear in the lowest (bass) voice?

We can expand this discussion to look at the scale systems found in the Billboard and RS-200 data sets using a topic model that identifies topics reflected in the chord annotations. When we visualize the scale-degree content found in those topics, distinct scale systems emerge. (Check out my late-breaking demo with Justin Glosson for further details! https://archives.ismir.net/ismir2021/latebreaking/000048.pdf)

In [ ]:
Image(filename='CPvsPop_Scales.png',width=900,height=500)